# Titanic dataset
* Get sample data [here](https://github.com/fuyu-quant/IBLM/tree/main/datasets).

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/IBLM.git@feature-in-context-learning 

In [2]:
import pkg_resources
print(pkg_resources.get_distribution('IBLM').version)

0.1.45


In [3]:
from dotenv import load_dotenv

# .env ファイルを読み込む
load_dotenv(override=True)

True

In [30]:
import pandas as pd
import numpy as np
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from iblm import IBLMClassifier

import sys
import os

In [90]:
n = 50

df = pd.read_csv(f'../data/titanic/titanic_{n}_train.csv')
x_train = df.drop('survived', axis=1)
y_train = df['survived']
print(len(x_train))

50


In [96]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model = llm_model, params=params)

### Training

In [92]:
file_path = '../models/titanic/'

model = iblm.fit(x_train, y_train, model_name = 'titanic', file_path=file_path)

Tokens Used: 7444
	Prompt Tokens: 7167
	Completion Tokens: 277
Successful Requests: 1
Total Cost (USD): $0.23162999999999997


In [ ]:
# Code of the model created
print(model)

import numpy as np

def predict(x):
    df = x.copy()
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        # Please describe the process required to make the prediction below.

        # The logic here is to give higher probability for those who are female, in first class, and embarked from Cherbourg
        # These are based on the known survival facts from the Titanic incident
        # The age, fare, and number of siblings/spouses/parents/children are also considered
        # The weights for each factor are determined based on their perceived impact on survival

        y = 0.3 * row['sex_female'] + 0.2 * row['class_First'] + 0.1 * row['embarked_C'] - 0.1 * row['age']/80 - 0.1 * row['fare']/500 - 0.1 * row['sibsp']/8 - 0.1 * row['parch']/6

        # The result is then passed through a sigmoid function to ensure the output is between 0 and 1
        y = 1 / (1 + np.exp(-y))

        # Do not change the code after this poin

### Prediction

In [81]:
df = pd.read_csv(f'../data/titanic/titanic_{n}_test.csv')
x_test = df.drop('survived', axis=1)
y_test = df['survived']

In [82]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [83]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.7164179104477612
Precision: 0.589098532494759
Recall: 0.8463855421686747
F1 score: 0.6946847960444994
ROC-AUC: 0.8317052998636476


### Creating Multiple Models

In [97]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {'columns_name': True}

iblm = IBLMClassifier(llm_model = llm_model, params=params)

In [98]:
seed = 3655
train_data = 300

train_num = [6,8,10,20,30,40,50]


for i in train_num:
    df = pd.read_csv(f'../data/titanic/titanic_{seed}_{train_data}_train.csv').head(i)
    x_train = df.drop('survived', axis=1)
    y_train = df['survived']

    file_path = '../models/titanic/'
    for j in range(1,31):
        model = iblm.fit(x_train, y_train, model_name = f'titanic_{seed}_{i}_{j}', file_path=file_path)

Tokens Used: 1634
	Prompt Tokens: 1343
	Completion Tokens: 291
Successful Requests: 1
Total Cost (USD): $0.057749999999999996


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


Tokens Used: 1659
	Prompt Tokens: 1343
	Completion Tokens: 316
Successful Requests: 1
Total Cost (USD): $0.05925
Tokens Used: 1741
	Prompt Tokens: 1343
	Completion Tokens: 398
Successful Requests: 1
Total Cost (USD): $0.06417
Tokens Used: 1612
	Prompt Tokens: 1343
	Completion Tokens: 269
Successful Requests: 1
Total Cost (USD): $0.05643
Tokens Used: 1663
	Prompt Tokens: 1343
	Completion Tokens: 320
Successful Requests: 1
Total Cost (USD): $0.05949
Tokens Used: 1665
	Prompt Tokens: 1343
	Completion Tokens: 322
Successful Requests: 1
Total Cost (USD): $0.059609999999999996
Tokens Used: 1616
	Prompt Tokens: 1343
	Completion Tokens: 273
Successful Requests: 1
Total Cost (USD): $0.05667
Tokens Used: 1616
	Prompt Tokens: 1343
	Completion Tokens: 273
Successful Requests: 1
Total Cost (USD): $0.05667
Tokens Used: 1625
	Prompt Tokens: 1343
	Completion Tokens: 282
Successful Requests: 1
Total Cost (USD): $0.05721
Tokens Used: 1659
	Prompt Tokens: 1343
	Completion Tokens: 316
Successful Requests:

In [ ]:
seed = 3656
train_data = 300

train_num = [6,8,10,20,30,40,50]


for i in train_num:
    df = pd.read_csv(f'../data/titanic/titanic_{seed}_{train_data}_train.csv').head(i)
    x_train = df.drop('survived', axis=1)
    y_train = df['survived']

    file_path = '../models/titanic/'
    for j in range(1,31):
        model = iblm.fit(x_train, y_train, model_name = f'titanic_{seed}_{i}_{j}', file_path=file_path)

In [ ]:
seed = 3657
train_data = 300

train_num = [6,8,10,20,30,40,50]


for i in train_num:
    df = pd.read_csv(f'../data/titanic/titanic_{seed}_{train_data}_train.csv').head(i)
    x_train = df.drop('survived', axis=1)
    y_train = df['survived']

    file_path = '../models/titanic/'
    for j in range(1,31):
        model = iblm.fit(x_train, y_train, model_name = f'titanic_{seed}_{i}_{j}', file_path=file_path)

### Prediction

In [ ]:
df = pd.read_csv(f'../data/titanic/titanic_{n}_test.csv')
x_test = df.drop('survived', axis=1)
y_test = df['survived']

sys.path.append('..')

error_count = 0
error_list = []
auc_list = []
acc_list = []


for i in range(1,31):

    import_file = f'import models.titanic.titanic_8_{i} as codemodel'

    exec(import_file)

    try:
        y_proba = codemodel.predict(x_test)
        y_pred = (y_proba > 0.5).astype(int)
        negative_values_exist = np.any(y_proba < 0)
        values_greater_than_one_exist = np.any(y_proba > 1)
        if negative_values_exist:
            error_list.append(i)
            error_count += 1
            print(f"Negative values exist：{negative_values_exist}")

        elif values_greater_than_one_exist:
            error_list.append(i)
            error_count += 1
            print(f"Positive values exist：{values_greater_than_one_exist}")

        else:
            roc_auc = roc_auc_score(y_test, y_proba)
            accuracy = round(accuracy_score(y_test, y_pred),4)
            auc_list.append(roc_auc)
            acc_list.append(accuracy)
        
    except Exception:
        print('run error')
        error_count += 1
        error_list.append(i)
        pass

print('-----------error-------------')
print(error_count)
print(error_list)

print('-----------AUC-------------')
print(auc_list)
average = sum(auc_list) / len(auc_list)
print("Average Value:", average)
max_value = max(auc_list)
min_value = min(auc_list)
print("Maximum Value:", max_value)
print("Minimum Value:", min_value)

print('-----------ACC-------------')
print(acc_list)
average = sum(acc_list) / len(acc_list)
print("Average Value:", average)
max_value = max(acc_list)
min_value = min(acc_list)
print("Maximum Value:", max_value)
print("Minimum Value:", min_value)

-----------error-------------
0
[]
-----------AUC-------------
[0.7781580804516585, 0.7201834862385321, 0.7011345746702135, 0.6221866348189565, 0.6571141631833235, 0.6347076705933445, 0.7459665599044569, 0.6207182020520059, 0.783798382281092, 0.6634438955539872, 0.5494679984799957, 0.32421964062754466, 0.6243797839422398, 0.783798382281092, 0.6320530915802616, 0.5802698007708593, 0.8802046577275936, 0.6046495847130993, 0.7011345746702135, 0.6634438955539872, 0.9127191791976549, 0.5522338635253243, 0.5295695130557516, 0.6953368438195539, 0.6634438955539872, 0.809323598067423, 0.7139758970739917, 0.5260599315998046, 0.5433798382281092, 0.7434422669779057]
Average Value: 0.6653505962397986
Maximum Value: 0.9127191791976549
Minimum Value: 0.32421964062754466
-----------ACC-------------
[0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.3828, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.6172, 0.7463